#DATAMINING PROJECT V1.8
***From Baptiste Danichert & Ahmed Abdel Aziz***

**Last update**: getting a better score

**Description:** You have noticed that to improve one’s skills in a new foreign language, it is important to read texts in that language. These texts have to be at the reader’s language level. However, it is difficult to find texts that are close to someone’s knowledge level (A1 to C2). You have decided to build a model for English speakers that predicts the difficulty of a French written text. This can be then used, e.g., in a recommendation system, to recommend texts, e.g, recent news articles that are appropriate for someone’s language level. If someone is at A1 French level, it is inappropriate to present a text at B2 level, as she won’t be able to understand it. Ideally, a text should have many known words and may have a few words that are unknown so that the person can improve.



# 1. Loading the training data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/BapDanSI/DataMiningProject/main/data/training_data.csv')
df_pred = pd.read_csv('https://raw.githubusercontent.com/BapDanSI/DataMiningProject/main/data/unlabelled_test_data.csv')

In [ ]:
df_pred.head()

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


In [ ]:
df.head()

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1


# 2. Dataframe analysis


In [ ]:
df.shape

(4800, 3)

-> 4800 rows and 2 columns (excluding first column "id")

In [ ]:
df.isnull().sum()

id            0
sentence      0
difficulty    0
dtype: int64

-> no NAs

In [ ]:
df.duplicated(subset="sentence").value_counts()

False    4800
dtype: int64

-> no duplicate in the data

#3.  Baseline

In [ ]:
np.random.seed = 0

In [ ]:
base_rate = max(df.value_counts('difficulty'))/df.shape[0]
print('Base rate:', round(base_rate,4))

Base rate: 0.1694


# 4. Tokinizer function

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!pip install -U wordstats
!python -m spacy download fr_core_news_sm
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stopwords
from wordstats import Word, common_words
from collections import Counter
import string
import nltk
import spacy
nlp = spacy.load("fr_core_news_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 8.7 MB/s 
     |████████████████████████████████| 1.2 MB 46.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for wordstats: filename=wordstats-1.0.7-py3-none-any.whl size=3616363 sha256=16d9b12258a28c4ee836fc203899bed6ab1c96b75f92ba5b3fa8af373f4dedf8
  Stored in directory: /root/.cache/pip/wheels/07/27/7d/24ade697c516ed02f369a90c5fec463286a6a67a299f75b711
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34527 sha256=3a9bd092d590abc5ff795592d5e998087aebf49033862c900c623f7575313b5d
  Stored in directory: /root/.cache/pip/wheels/e0/ea/e1/7b17c0af6f0228de33d82556dbad4fe823d90696fc50f2c81d
Successfully built wordstats configobj
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWar

In [ ]:
df['n_diff'] = df['difficulty'].replace(['A1', 'A2', 'B1', 'B2', 'C1', 'C2'], [1,2,3,4,5,6])

In [ ]:
punctuations = string.punctuation
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
stop_words=spacy.lang.fr.stop_words.STOP_WORDS

# Create tokenizer function
def spacy_tokenizer(sentence):
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in mytokens ]


    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Return preprocessed list of tokens
    return mytokens

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer) # we use the above defined tokenizer

#5. Improve text preparation

In [ ]:
# Create list of configs
def configs():

    models = list()
    
    # Define config lists
    ngram_range = [(1,1), (1,2), (1, 3), (2, 2), (2, 3), (3, 3)]
    min_df = [1]
    max_df = [1.0]
    analyzer=['word', 'char']
    
    # Create config instances
    for n in ngram_range:
        for i in min_df:
            for j in max_df:
              for a in analyzer:
                    cfg = [n, i, j, a]
                    models.append(cfg)
    return models

configs = configs()
configs[:10]

[[(1, 1), 1, 1.0, 'word'],
 [(1, 1), 1, 1.0, 'char'],
 [(1, 2), 1, 1.0, 'word'],
 [(1, 2), 1, 1.0, 'char'],
 [(1, 3), 1, 1.0, 'word'],
 [(1, 3), 1, 1.0, 'char'],
 [(2, 2), 1, 1.0, 'word'],
 [(2, 2), 1, 1.0, 'char'],
 [(2, 3), 1, 1.0, 'word'],
 [(2, 3), 1, 1.0, 'char']]

In [ ]:

# Define list for result
result = []

for config in configs:

    # Redefine vectorizer
    tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer, 
                                   ngram_range=config[0],
                                   min_df=config[1], max_df=config[2], analyzer=config[3])

# 6. Classification Algorithms

Dependent variable (y) is the column named "difficulty".
<br>We split the data into 80% training and 20% test set.


In [ ]:
y = df['difficulty']
X = df["sentence"]
X_pred = df_pred["sentence"]

### i. Logistic Regression.
We use the following parameters for the LogisticRegressionCV():

* cross-validation to 5 folds
* maximum interation to 1000
* random state to 0

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def evaluate(true, pred):
    precision = precision_score(true, pred,average='macro')
    recall = recall_score(true, pred,average='macro')
    f1 = f1_score(true, pred,average='macro')
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [ ]:
np.random.seed = 0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=50)

classifier = LogisticRegressionCV()

pl = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

pl.fit(X_train, y_train)

y_pred = pl.predict(X_test)

print("CONFIG: ", config)
evaluate(y_test, y_pred)
print("-----------------------")

result.append([config, accuracy_score(y_test, y_pred)])

CONFIG:  [(3, 3), 1, 1.0, 'char']
CONFUSION MATRIX:
[[60 22  6  0  1  0]
 [18 38 22  2  0  1]
 [ 9 26 27  4  3  4]
 [ 3  2  7 39 14 17]
 [ 1  2  9  9 38 22]
 [ 2  1  6 13 12 40]]
ACCURACY SCORE:
0.5042
CLASSIFICATION REPORT:
	Precision: 0.5051
	Recall: 0.4997
	F1_Score: 0.5002
-----------------------


In [ ]:
predict = pl.predict(X_pred)

In [ ]:
submission= pd.DataFrame()
submission['id']= df_pred.index
submission['difficulty'] = predict

In [ ]:
submission.to_csv("submission.csv", index=False)

### ii. kNNeighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

grid = {'n_neighbors':np.arange(1,100),
        'p':np.arange(1,3),
        'weights':['uniform','distance']}

knn = KNeighborsClassifier()
classifier_knn = GridSearchCV(knn, grid, cv=5)

pl_knn = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier_knn)])

pl_knn.fit(X_train, y_train)

print("Hyperparameters:", classifier_knn.best_params_)

In [ ]:
y_knn_predict = pl_knn.predict(X_test)
evaluate(y_test, y_knn_predict)

CONFUSION MATRIX:
[[61 18  8  1  0  1]
 [25 30 14  5  4  3]
 [19 22 25  3  1  3]
 [ 6  6 10 29 17 14]
 [ 4  4  5  6 29 33]
 [ 5  2  4  6 10 47]]
ACCURACY SCORE:
0.4604
CLASSIFICATION REPORT:
	Precision: 0.4623
	Recall: 0.4575
	F1_Score: 0.4494


In [ ]:
knn_predict = pl_knn.predict(X_pred)
submission_knn= pd.DataFrame()
submission_knn['id']= df_pred.index
submission_knn['difficulty'] = knn_predict
submission_knn.to_csv("submissionknn.csv", index=False)

###  iii. Decision Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
pl_dtc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', dtc)])
pl_dtc.fit(X_train, y_train)
y_pred_dtc = pl_dtc.predict(X_test)

evaluate(y_test, y_pred_dtc)

CONFUSION MATRIX:
[[51 20  6  7  5  0]
 [23 24 17  8  4  5]
 [11 13 21  9 13  6]
 [ 7  9  9 30 13 14]
 [ 2  5 13 17 24 20]
 [ 5  3 12  7 27 20]]
ACCURACY SCORE:
0.3542
CLASSIFICATION REPORT:
	Precision: 0.3467
	Recall: 0.3482
	F1_Score: 0.3468


In [ ]:
dtc_predict = pl_dtc.predict(X_pred)
submission_dtc= pd.DataFrame()
submission_dtc['id']= df_pred.index
submission_dtc['difficulty'] = dtc_predict
submission_dtc.to_csv("submissiondtc.csv", index=False)

### iv. Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
pl_rfc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', rfc)])

pl_rfc.fit(X_train, y_train)

y_pred_rfc = pl_rfc.predict(X_test)

evaluate(y_test, y_pred_rfc)

CONFUSION MATRIX:
[[68 13  3  4  0  1]
 [23 30 22  6  0  0]
 [ 7 24 28  5  5  4]
 [ 4  5  8 29 22 14]
 [ 1  5  5 15 37 18]
 [ 2  1  4 13 21 33]]
ACCURACY SCORE:
0.4688
CLASSIFICATION REPORT:
	Precision: 0.4570
	Recall: 0.4624
	F1_Score: 0.4585


In [ ]:
rfc_predict = pl_rfc.predict(X_pred)
submission_rfc= pd.DataFrame()
submission_rfc['id']= df_pred.index
submission_rfc['difficulty'] = rfc_predict
submission_rfc.to_csv("submissionrfc.csv", index=False)

### v. Support Vector Classifier

In [ ]:
from sklearn.svm import SVC

svc = SVC()
pl_svc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', svc)])

pl_svc.fit(X_train, y_train)

y_pred_svc = pl_svc.predict(X_test)

evaluate(y_test, y_pred_svc)

CONFUSION MATRIX:
[[62 19  7  0  1  0]
 [22 36 22  1  0  0]
 [ 7 21 35  6  1  3]
 [ 2  1  8 48 14  9]
 [ 1  3  5 16 37 19]
 [ 2  1  7 13 12 39]]
ACCURACY SCORE:
0.5354
CLASSIFICATION REPORT:
	Precision: 0.5341
	Recall: 0.5316
	F1_Score: 0.5312


In [ ]:
svc_predict = pl_svc.predict(X_pred)
submission_svc= pd.DataFrame()
submission_svc['id']= df_pred.index
submission_svc['difficulty'] = svc_predict
submission_svc.to_csv("submissionsvc.csv", index=False)

###vi. AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier()
pl_abc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', abc)])

pl_abc.fit(X_train, y_train)

y_pred_abc = pl_abc.predict(X_test)

evaluate(y_test, y_pred_abc)

CONFUSION MATRIX:
[[56 24  3  4  0  2]
 [27 28 21  3  0  2]
 [12 18 21  9  5  8]
 [ 3  6 12 31 16 14]
 [ 1  4  9  8 34 25]
 [ 0  2  2 17 12 41]]
ACCURACY SCORE:
0.4396
CLASSIFICATION REPORT:
	Precision: 0.4333
	Recall: 0.4357
	F1_Score: 0.4322


In [ ]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier()
pl_mlpc = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', mlpc)])

pl_mlpc.fit(X_train, y_train)

y_pred_mlpc = pl_mlpc.predict(X_test)

evaluate(y_test, y_pred_mlpc)

CONFUSION MATRIX:
[[50 32  6  0  1  0]
 [18 38 21  2  1  1]
 [11 21 31  3  2  5]
 [ 2  5  9 34 15 17]
 [ 1  1 10  8 41 20]
 [ 1  1  8 18 13 33]]
ACCURACY SCORE:
0.4729
CLASSIFICATION REPORT:
	Precision: 0.4790
	Recall: 0.4704
	F1_Score: 0.4722


In [ ]:
mlpc_predict = pl_mlpc.predict(X_pred)
submission_mlpc= pd.DataFrame()
submission_mlpc['id']= df_pred.index
submission_mlpc['difficulty'] = mlpc_predict
submission_mlpc.to_csv("submissionmlpc.csv", index=False)

#7. Comparing models

In [ ]:
import pandas as pd

df_comparison = pd.DataFrame(columns=['Model_Name', 'Base_Rate', 'Precisions','Recall','F1-Score', 'Accuracy'])


df_comparison['Model_Name'] = ['Logistic Reg', 'KNN', 'Tree', 'Random Forest', 'Support Vector', 'AdaBooster', 'MLPC']
df_comparison['Base_Rate'] = ['0.1694', '0.1694', '0.1694', '0.1694', '0.1694', '0.1694', '0.1694']
df_comparison['Precisions'] = ['0.5051', '0.4623', '0.3467','0.4570', '0.5341', '0.4333', '0.4790']
df_comparison['Recall'] = ['0.4997', '0.4575', '0.3482','0.4624', '0.5316', '0.4357', '0.4704']
df_comparison['F1-Score'] = ['0.5002', '0.4494', '0.3468','0.4585', '0.5312', '0.4322', '0.4722']
df_comparison['Accuracy'] = ['0.5042', '0.4604', '0.3542','0.4688', '0.5354', '0.4396', '0.4729']


df_comparison